## Using the RNC to look at preps in certain windows of verbs

In [1]:
import random

In [2]:
from corus import load_morphoru_rnc

path = 'RNCgoldInUD_Morpho.conll'
records = load_morphoru_rnc(path)
rnccorpus = []
for record in records:
    rnccorpus.append(record)
len(rnccorpus)

98892

## prep + case!!!

In [10]:
# def build_sentence(tokens, upperindexlist):
#     sentence = ''
#     for i, token in enumerate(tokens):
#         if token.pos == 'PUNCT':
#             space = ''
#         else:
#             space = ' '
#         if i in upperindexlist:
#             text = token.text.upper()
#         else:
#             text = token.text
#         if text == None:
#             text = ''
#         sentence = sentence + space + text
#     return sentence.strip()

In [34]:
def build_sentence(tokens, upperlist):
    sentence = ''
    for i, token in enumerate(tokens):
        if token.pos == 'PUNCT':
            space = ''
        else:
            space = ' '
        if i in upperlist:
            text = token.text.upper()
        else:
            text = token.text
        if text == None:
            text = ''
        sentence = sentence + space + text
    return sentence.strip()

# sample = random.sample(rnccorpus,100)
sample = rnccorpus
windowList = []
for window in range(-3, 4):
    dataDict = dict()
    for sent in sample:
        currentindex = 0
        for token in sent.tokens:
            if token.pos == 'VERB':
                verblemma = token.lemma
#                 if verblemma not in okayverbs:
#                     break
                if verblemma not in dataDict:
                    dataDict[verblemma] = dict()
                    dataDict[verblemma]['counts'] = dict()
                    dataDict[verblemma]['sentences'] = dict()
                adpindex = currentindex + window
                if adpindex >= 0 and adpindex < len(sent.tokens):
                    adptoken = sent.tokens[adpindex]
                    if adptoken.pos == 'PUNCT' and adpindex + 1 >= 0 and adpindex + 1 < len(sent.tokens):
                        adpindex = adpindex + 1
                    if adptoken.pos == 'ADP':
                        adplemma = adptoken.lemma
                        caseindex = adpindex + 1
    
                        if caseindex >= 0 and caseindex < len(sent.tokens):
                            casetoken = sent.tokens[caseindex]
                            if casetoken.pos == 'PUNCT' and caseindex + 1 >= 0 and caseindex + 1 < len(sent.tokens):
                                caseindex = caseindex + 1
                            casetoken = sent.tokens[caseindex]
                            try:
                                case = casetoken.feats['Case']
                                if case == 'Ins':
                                    case = 'Inst'
                                adpandcase = adplemma + ' + ' + case.upper()
                            except:
                                break
                                
                    else:
                        if window in [-1,1]:
                            # not an adposition directly before or after verb
                            try: 
                                if adpindex - 1 >= 0:
                                    # need to make sure prev token is not adp
                                    if sent.tokens[adpindex - 1].pos != 'ADP':
                                        case = adptoken.feats['Case']
                                        if case == 'Ins':
                                            case = 'Inst'
                                        adpandcase = 'No prep + ' + case.upper()
                                        caseindex = currentindex
                                    else:
                                        break
                                else:
                                    break
                            except:
                                break
                        else:
                            break
                            
                    if adpandcase not in dataDict[verblemma]['counts']:                               
                        dataDict[verblemma]['counts'][adpandcase] = 0
                        dataDict[verblemma]['sentences'][adpandcase] = []

                    dataDict[verblemma]['counts'][adpandcase] += 1
                    formattedsent = build_sentence(sent.tokens, [currentindex, adpindex, caseindex])
                    if formattedsent not in dataDict[verblemma]['sentences'][adpandcase]:
                        dataDict[verblemma]['sentences'][adpandcase].append(formattedsent)
                                
                    
            currentindex += 1
    print(f'window size of {window} complete...')
    windowList.append(dataDict)

window size of -3 complete...
window size of -2 complete...
window size of -1 complete...
window size of 0 complete...
window size of 1 complete...
window size of 2 complete...
window size of 3 complete...


In [33]:
def build_sentence(tokens):
    sentence = ''
    for i, token in enumerate(tokens):
        if token.pos == 'PUNCT':
            space = ''
        else:
            space = ' '
        text = token.text
        if text == None:
            text = ''
        sentence = sentence + space + text
    return sentence.strip()

for sent in rnccorpus:
    thissent = build_sentence(sent.tokens).lower()
    if ' с ХОДУ ОСТАНОВИЛАСЬ, как будто на столб наткнулась.'.lower() in thissent:
        print(sent)

MorphoSent(tokens=[MorphoToken(text='--', lemma='--', pos='PUNCT', feats={}, feats2={}), MorphoToken(text='Анна', lemma='Анна', pos='NOUN', feats={'Animacy': 'Anim', 'Case': 'Nom', 'Gender': 'Fem', 'Number': 'Sing'}, feats2={'NameType': 'Giv'}), MorphoToken(text='Фёдоровна', lemma='Федоровна', pos='NOUN', feats={'Animacy': 'Anim', 'Case': 'Nom', 'Gender': 'Fem', 'Number': 'Sing'}, feats2={'NameType': 'Ptr'}), MorphoToken(text='с', lemma='с', pos='ADP', feats={}, feats2={}), MorphoToken(text='ходу', lemma='ход', pos='NOUN', feats={'Animacy': 'Inan', 'Case': 'Gen', 'Gender': 'Masc', 'Number': 'Sing'}, feats2={}), MorphoToken(text='остановилась', lemma='остановиться', pos='VERB', feats={'Gender': 'Fem', 'Mood': 'Ind', 'Number': 'Sing', 'Tense': 'Past', 'VerbForm': 'Fin', 'Voice': 'Mid'}, feats2={'Subcat': 'Intr', 'Aspect': 'Perf'}), MorphoToken(text=',', lemma=',', pos='PUNCT', feats={}, feats2={}), MorphoToken(text='как', lemma='как', pos='ADV', feats={'Degree': 'Pos'}, feats2={}), Morph

In [41]:
windowList[2]['остановиться']['sentences']['No prep + GEN']

['На анализе методического аппарата при изучении смысла жизни УЧИТЕЛЯ ОСТАНОВИЛАСЬ Е. А. Максимова( Москва) в сообщении" Комплекс экспериментальных методик изучения смысложизненных ориентаций".']

In [42]:
windowList[2]['сходить']['sentences']['No prep + ACC']

['А когда первый РАЗ СХОДИЛА в кино и увидела музыкантов, которые играют в фойе, мне стало жаль моего бедного дедушку: зрители переговаривались, жевали бутерброды, шуршали газетами, а старые люди на сцене играли вальс.',
 'Наш корреспондент Микаэль Укин получил ЗАДАНИЕ СХОДИТЬ в Мавзолей.']

In [43]:
# removing uncommon verbs (<10 occurrences)
uncommonVerbs = set()
totalcount = dict()
for windowDict in windowList:
    for verb in windowDict:
        if verb not in totalcount:
            totalcount[verb] = 0
        countDict = windowDict[verb]['counts']
        for prep in windowDict[verb]['counts']:
            totalcount[verb] += windowDict[verb]['counts'][prep]
for verb in totalcount:
    if totalcount[verb] < 30:
        uncommonVerbs.add(verb)
print(len(uncommonVerbs))

7678


In [44]:
for verbtoremove in uncommonVerbs:
    for windowDict in windowList:
        try:
            del windowDict[verbtoremove]
        except:
            False

In [45]:
len(windowList[1])

499

In [46]:
windowList[1]['читать']['counts']

{'в + LOC': 4,
 'среди + GEN': 1,
 'на + LOC': 1,
 'с + INST': 2,
 'из + GEN': 1,
 'после + GEN': 1,
 'по + DAT': 1}

In [47]:
pwd

'/Users/paigelee/Desktop/spring2021/clancy/verbhistograms'

## for csv

In [48]:
allcxlabels = set()
for windowDict in windowList:
    for verb in windowDict:
        countdict = windowDict[verb]['counts']
        for cxlabel in countdict:
            allcxlabels.add(cxlabel)
allcxlabels = list(allcxlabels)

In [49]:
# getting rid of cxx with count < 10
totalcxcounts = dict()
for windowDict in windowList:
    for verb in windowDict:
        countdict = windowDict[verb]['counts']
        for cx in countdict:
            if cx not in totalcxcounts:
                totalcxcounts[cx] = 0
            totalcxcounts[cx] += 1
lst = []
for cx in totalcxcounts:
    lst.append((totalcxcounts[cx],cx))
i = 0
greaterthan10cx = []
for ct, cx in lst:
    if ct > 10:
        greaterthan10cx.append(cx)

In [50]:
# verb, windowsize, prep1, prep2, prep3, prep4, prep5...
rows = []
# cxused = allcxlabels
cxused = greaterthan10cx
for windowIndex, windowDict in enumerate(windowList):
    windowSize = windowIndex - 3
    for verb in windowDict:
        countdict = windowDict[verb]['counts']
        csvline = [verb, windowSize]
        for cxlabel in cxused:
            if cxlabel in countdict:
                count = countdict[cxlabel]
            else:
                count = 0
            csvline.append(count)
        rows.append(csvline)

In [51]:
r = random.choice(rows)
print(r)

['являться', -3, 12, 3, 4, 1, 2, 2, 1, 4, 1, 0, 0, 0, 1, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [52]:
fields = ['Verb','WindowSize']

for cxlabel in greaterthan10cx:
    fields.append(cxlabel)

In [53]:
pwd

'/Users/paigelee/Desktop/spring2021/clancy/verbhistograms'

In [54]:
import csv
# writing to csv file 
filename = 'csvdata1.csv'
with open(filename, 'w') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile) 
        
    # writing the fields 
    csvwriter.writerow(fields) 
        
    # writing the data rows 
    csvwriter.writerows(rows)

In [33]:
cd verbhistograms/

/Users/paigelee/Desktop/spring2021/clancy/verbhistograms


In [55]:
sentenceDict = dict()
windowSize = -3
for windowDict in windowList:
    sentenceDict[windowSize] = dict()
    for verb in windowDict:
        sentenceDict[windowSize][verb] = dict()
        for cx in windowDict[verb]['sentences']:
            sentenceDict[windowSize][verb][cx] = windowDict[verb]['sentences'][cx][:10]
    windowSize += 1

In [56]:
totalsents = 0
for windowDict in windowList:
    for verb in windowDict:
        for prep in windowDict[verb]['sentences']:
            totalsents += len(windowDict[verb]['sentences'][prep])
print(totalsents)

63524


In [57]:
verbDict = dict()

for verb in windowList[0]:
    verbDict[verb] = dict()
windowSize = -3
for windowDict in windowList:
    for verb in windowDict:
        verbDict[verb][windowSize] = dict()
        for cx in windowDict[verb]['sentences']:
            verbDict[verb][windowSize][cx] = list()
            for sent in windowDict[verb]['sentences'][cx]:
                verbDict[verb][windowSize][cx].append(sent)
    windowSize += 1

In [58]:
verbDict['помочь'][1].keys()

dict_keys(['No prep + DAT', 'No prep + ACC', 'No prep + NOM', 'No prep + INST'])

In [60]:
import json
for verb in verbDict:
    with open(f'updatedsentdata/{verb}.json', 'w', encoding='utf8') as jsonfile:
        json.dump(verbDict[verb], jsonfile, ensure_ascii=False)

### write sentence dictionary to json

In [78]:
# writing dictionary to json

import json

filepath = 'sentences.json'

with open(filepath, 'w', encoding='utf8') as json_file:
    json.dump(windowList, json_file, ensure_ascii=False)
    
print(filepath,'written successfully.')

sentences.json written successfully.
